# import

In [317]:
import numpy as np
import pandas as pd
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [318]:
# фиксируем RANDOM_SEED
RANDOM_SEED = 42

In [319]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [320]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [321]:
df_train.info()

In [322]:
df_train.head(5)

In [323]:
df_test.info()

In [324]:
df_test.head(5)

In [325]:
sample_submission.head(5)

In [326]:
sample_submission.info()

In [327]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [328]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [329]:
data.sample(5)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

### 1. Cleaning and Prepping Data

In [330]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Cuisine Style'] = data['Cuisine Style'].fillna('0') # replace empty cells with "0"
data['Price Range'] = data['Price Range'].fillna('0') # replace empty cells with "0"
data['Number of Reviews'] = data['Number of Reviews'].fillna(1) # replace empty cells with 0
data['Reviews'] = data['Reviews'].fillna('0') # replace empty cells with "0"

### 2. Обработка признаков


#### Возьмем следующий признак "Price Range".

In [331]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [332]:
# create column of quantitative expression of price
def prices(price):
    if price == '$':
        return 1
    if price == '$$ - $$$':
        return 2
    if price == '$$$$':
        return 3
    else: return None

In [333]:
data['Prices'] = data['Price Range'].apply(prices)

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

In [334]:
# adding data of population in cities
population = {
        'London': 9046,
        'Paris': 10901,
        'Madrid': 6497,
        'Barcelona': 5494,
        'Berlin': 3552,
        'Milan': 3132,
        'Rome': 4210,
        'Prague': 1292,
        'Lisbon': 2927,
        'Vienna': 1901,
        'Amsterdam': 1132,
        'Brussels': 2050,
        'Hamburg': 1793,
        'Munich': 1504,
        'Lyon': 1690,
        'Stockholm': 1583,
        'Budapest': 1759,
        'Warsaw': 1768,
        'Dublin': 1201,
        'Copenhagen': 1321,
        'Athens': 3156,
        'Edinburgh': 525,
        'Zurich': 1371,
        'Oporto': 1307,
        'Geneva': 599,
        'Krakow': 767,
        'Oslo': 1012,
        'Helsinki': 1279,
        'Bratislava': 430,
        'Luxembourg': 119,
        'Ljubljana': 292
    }
data['Population'] = data['City'].map(population)

In [335]:
capitals = [
        'London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon',
        'Vienna', 'Amsterdam', 'Brussels', 'Stockholm', 'Budapest', 'Warsaw',
        'Dublin', 'Copenhagen', 'Athens', 'Oslo', 'Helsinki', 'Bratislava',
        'Luxembourg', 'Ljubljana', 'Edinburgh'
    ]
data['Capital'] = data['City'].apply(lambda x: 1 if x in capitals else 0)

In [336]:
country = {
        'London': 'GBR',
        'Paris': 'FRA',
        'Madrid': 'ESP',
        'Barcelona': 'ESP',
        'Berlin': 'GER',
        'Milan': 'ITA',
        'Rome': 'ITA',
        'Prague': 'CZE',
        'Lisbon': 'POR',
        'Vienna': 'AUS',
        'Amsterdam': 'NED',
        'Brussels': 'BEL',
        'Hamburg': 'GER',
        'Munich': 'GER',
        'Lyon': 'FRA',
        'Stockholm': 'SWE',
        'Budapest': 'HUN',
        'Warsaw': 'POL',
        'Dublin': 'IRE',
        'Copenhagen': 'DEN',
        'Athens': 'GRE',
        'Edinburgh': 'GBR',
        'Zurich': 'CHE',
        'Oporto': 'POR',
        'Geneva': 'CHE',
        'Krakow': 'POL',
        'Oslo': 'NOR',
        'Helsinki': 'FIN',
        'Bratislava': 'SVK',
        'Luxembourg': 'LUX',
        'Ljubljana': 'SVN'
    }

data['Country'] = data['City'].map(country)

In [337]:
# replacing string by list in Cuisine Style 
def cuisinename(cuisine):
    cuisine = cuisine.replace('[','')
    cuisine = cuisine.replace(']', '')
    cuisine = cuisine.replace(' ','')     
    k = list(cuisine.split(','))
    return(k)

# calculating number of cuisines in restraunt 
def cuisine(cuisine):
    if cuisine[0] == '0':
        return 0
    else: return len(cuisine)

In [338]:
data['Cuisine Style'] = data['Cuisine Style'].apply(cuisinename) # applying changing to column
data['cuisines'] = data['Cuisine Style'].apply(cuisine) # creating columns of number of cuisines in restaurant

In [339]:
# create list of dates of reviews
def review_dates(review):
    found_date = re.findall(r'\d{2}/\d{2}/\d{4}', review)
    return found_date

data['Dates of Reviews'] = data["Reviews"].apply(review_dates)

In [340]:
#create difference between reviews
def difference(diff):
    try:
        k = pd.to_datetime(diff, format='%m/%d/%Y')
        s = max(k) - min(k)
        return(s.days)
    except:
        return None

data['Difference between Reviews'] = data["Dates of Reviews"].apply(difference)        

In [341]:
data['Difference between Reviews'] = data['Difference between Reviews'].fillna(data['Difference between Reviews'].mean()) # replace empty cells with mean  
data['Prices'] = data['Prices'].fillna(data.Prices.mean()) # replace empty cells with mean 

In [342]:
#creating columns with coefficient of importance of review on timeline
def differ(diff):
    try:
        k = pd.to_datetime(diff)
        s = max(k)
        return(365/(datetime.datetime.now() - s).days)
    except:
        return None
    
data['Review Importance'] = data["Dates of Reviews"].apply(differ)
data['Review Importance'] = data['Review Importance'].fillna(0.0001) # replace empty cells

In [343]:
data['Rating_per_Price'] = data['Rating']/data['Prices']
data['Reviews per Population'] = data['Population']/(data['Number of Reviews']+0.01)

# EDA 

### Посмотрим распределение признака

In [344]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [345]:
df_train['City'].value_counts(ascending=True).plot(kind='bar')

Посмотрим, как изменится распределение в большом городе:

In [346]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [347]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.


### Посмотрим распределение целевой переменной

In [348]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [349]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [350]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Dummies

In [351]:
data = pd.get_dummies(data, columns=['City',])
data = pd.get_dummies(data, columns=['Country',])
data2 = data['Cuisine Style'].apply(lambda x: pd.Series([1]*len(x),index =x)).fillna(0, downcast = 'infer')
data = data.join(data2, how = 'outer')

### корреляция признаков

In [352]:
correl = data.corr().abs() # checking correlation


s = correl.unstack()
so = s.sort_values(kind="quicksort")
so[(so > 0.75) & (so < 1)]

### Удаление лишних столбцов

In [353]:
# drop high-correlated columns
df = data.drop(['Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 'Dates of Reviews','Rating_per_Price','Country_POL', 'City_Lisbon', 'City_Paris','City_London'], axis = 1)

#### Запускаем и проверяем что получилось

In [354]:
df.describe()

In [355]:
# Теперь выделим тестовую часть
train_data = df.query('sample == 1').drop(['sample'], axis=1)
test_data = df.query('sample == 0').drop(['sample'], axis=1)

y = train_data['Rating']            # наш таргет
X = train_data.drop(['Restaurant_id','Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [356]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 25% данных на валидацию (параметр test_size)
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [357]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [358]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели


In [359]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [360]:
# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [361]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [362]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [363]:
test_data.sample(10)

In [364]:
test_data = test_data.drop(['Restaurant_id','Rating'], axis=1)

In [365]:
sample_submission

In [367]:
predict_submission = regr.predict(test_data)

In [368]:
predict_submission

In [369]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)